In [1]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import random
import zipfile
device='cpu'

In [2]:
def load_data_jay_lyrics():
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars=f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([( char,i ) for i , char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [3]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

In [4]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)
rnn_layer2=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens,num_layers=2)
rnn_layer

RNN(1027, 256)

In [5]:
rnn_layer2

RNN(1027, 256, num_layers=2)

In [6]:
num_steps=35
batch_size=2
state=None
X=torch.rand(num_steps,batch_size,vocab_size)
Y,state_new=rnn_layer(X,state)
print(X.shape)
print(Y.shape,len(state_new),state_new[-1].shape)

torch.Size([35, 2, 1027])
torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [1]:
def to_onehot(x,n_class):
    def one_hot(x,n_class):
        x=x.long()
        res=torch.zeros(x.shape[0],n_class)
        res=res.scatter(1,x.view(-1,1),1)
        return res
    return [one_hot(x[:,i],n_class) for i in range(x.shape[1])]


In [2]:
x=torch.arange(10).view(2,5)
to_onehot(x,10)

NameError: name 'torch' is not defined

In [9]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super().__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,self.vocab_size)
        self.state=None
        
    def forward(self,inputs,state):
        X=to_onehot(inputs,self.vocab_size)
        Y,self.state=self.rnn(torch.stack(X),state)
        output=self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state
        

In [30]:
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,device,idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix)-1):
#         print(output[-1])
        X=torch.Tensor([output[-1]]).view(1,1)
        if state is not None:
            if isinstance(state,tuple):
                state=(state[0],state[1])
            else:
                state=state
        (Y,state)=model(X,state)
        if t < len(prefix)-1 :
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])
        

In [31]:
model=RNNModel(rnn_layer,vocab_size)
predict_rnn_pytorch('分开',10,model,vocab_size,device,idx_to_char,char_to_idx)

'分开公难夜公话忧著歌努将'

In [32]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.Tensor(corpus_indices,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y

In [33]:
def grad_clipping(params,theta,device):
    norm=torch.Tensor([0.0])
    for p in params:
        norm+=(p.grad.data **2).sum()
    norm=norm.sqrt().item()
    if norm > theta:
        for p in params:
            p.grad.data *= (theta/norm)

In [34]:
def train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,num_epochs,num_steps,lr,
                                 clipping_theta,batch_size,pred_period,pred_len,prefixes):
    loss=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state=None
    for epoch in range(num_epochs):
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_consecutive(corpus_indices,batch_size,num_steps)
        for X,Y in data_iter:
            if state is not None :
                if isinstance(state,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
            (output,state)=model(X,state)
            Y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(output,Y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(),clipping_theta,device)
            
            optimizer.step()
            l_sum +=l.item() * Y.shape[0]
            n+=Y.shape[0]
            
            try:
                perplexity = math.exp(l_sum/n)
            except OverflowError:
                perplexity=float('inf')
            if (epoch+1) % pred_period ==0:
                print('epoch %d ,perplexity %f ,tim %.2f sec' % (epoch +1,perplexity,time.time()-start))
                
                for prefix in prefixes:
                    print('- ',predict_rnn_pytorch(prefix,pred_len,model,vocab_size,device,idx_to_char,char_to_idx))
            
            

In [35]:
num_steps=35
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                              corpus_indices, idx_to_char, char_to_idx,
                              num_epochs, num_steps, lr, clipping_theta,
                              batch_size, pred_period, pred_len, prefixes)

epoch 50 ,perplexity 1.202714 ,tim 0.12 sec
-  分开 你要再 小打我妈爸 一只两步三分四步 有天  是再人的你 著一直在我说 一定实现 在人开 快使上双
-  不分开 我 我的到 可爱你在 打来的可爱 我不要 想要你却只笑每天叹手 到底说的我想要 一着两听 回的像 
epoch 50 ,perplexity 1.169291 ,tim 0.24 sec
-  分开 你要再 小打我妈爸 一只两步三分四步 有天  是再人的你 著一直在我说 一定实现 在人开始痛久 回
-  不分开 我 我的到 可爱你在 打来的可爱 我不要不想  我想要你想很  一个秋 后知后觉 我该好好生活 我
epoch 50 ,perplexity 1.154009 ,tim 0.42 sec
-  分开 你要再 小打我 爸想一你 手不能 平常话不多 除非是乌鸦抢了它的窝 它在灌木丛旁邂逅 一只令它心仪
-  不分开 我 我的到 可爱你在 打来的可爱 我不要不想  我想要你想很  一个  后怎么会 手不要 想然到在
epoch 50 ,perplexity 1.152706 ,tim 0.57 sec
-  分开 你要再 已打我 别想就  什么 干跟你 我不懂 想你这样 我不的 爱情来的太多 像想你这样打我妈妈
-  不分开 我 我的到 可爱你在 打来的可爱 我不要不想  我想要你想很  一个  后怎么会 手不要 想然到在
epoch 50 ,perplexity 1.152904 ,tim 0.70 sec
-  分开 你要再 已打我 别想就  什么 干跟你 我不懂 想你这样 我不的我爱你的让我 不的 爱女人 漂亮的
-  不分开 我 我的到 可爱你在 打来的可爱 我不要不想  我想要你想很  一个  后怎么会 手不要 想然到在
epoch 50 ,perplexity 1.151290 ,tim 0.83 sec
-  分开 你要再 已打我 别想就  什么 干跟你 我不懂 想你这样 我不的我爱你的让我 不的 爱女人 漂亮的
-  不分开 我 我的到 可爱你在 打来的可爱 我不要不想  我想要你想很  一个秋 后知后觉 我该好好生活 我
epoch 50 ,perplexity 1.147198 ,tim 1.12 sec
-  分开 你要再 小打我 